In [ ]:
!git clone https://github.com/yachty66/SimSwap.git
!cd SimSwap && git pull

In [ ]:
!pip install insightface==0.2.1 onnxruntime moviepy
!pip install googledrivedownloader
!pip install imageio==2.4.1

In [ ]:
import os
os.chdir("SimSwap")
!ls

In [ ]:
from google_drive_downloader import GoogleDriveDownloader

### it seems that google drive link may not be permenant, you can find this ID from our open url.
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N',
#                                     dest_path='./arcface_model/arcface_checkpoint.tar')
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1PXkRiBUYbu1xWpQyDEJvGKeqqUFthJcI',
#                                     dest_path='./checkpoints.zip')

!wget -P ./arcface_model https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!unzip ./checkpoints.zip  -d ./checkpoints
!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth

In [ ]:
!wget --no-check-certificate "https://sh23tw.dm.files.1drv.com/y4mmGiIkNVigkSwOKDcV3nwMJulRGhbtHdkheehR5TArc52UjudUYNXAEvKCii2O5LAmzGCGK6IfleocxuDeoKxDZkNzDRSt4ZUlEt8GlSOpCXAFEkBwaZimtWGDRbpIGpb_pz9Nq5jATBQpezBS6G_UtspWTkgrXHHxhviV2nWy8APPx134zOZrUIbkSF6xnsqzs3uZ_SEX_m9Rey0ykpx9w" -O antelope.zip
!unzip ./antelope.zip -d ./insightface_func/models/

In [ ]:
import cv2
import torch
import fractions
import numpy as npa
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from models.models import create_model
from options.test_options import TestOptions
from insightface_func.face_detect_crop_multi import Face_detect_crop
from util.videoswap_multispecific import video_swap
import os
import glob

In [ ]:
def lcm(a, b): return abs(a * b) / fractions.gcd(a, b) if a and b else 0

transformer = transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

transformer_Arcface = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [ ]:
!ls ./checkpoints

In [ ]:
!pip install opencv-python
!apt install cmake
!pip install face_recognition

In [ ]:
import os
import cv2
import numpy as np
import onnxruntime
import requests
from tqdm import tqdm
import face_recognition


# Define the model URL and local path
model_url = 'https://github.com/facefusion/facefusion-assets/releases/download/models/yoloface_8n.onnx'
model_path = './models/yoloface_8n.onnx'

# Ensure the directory exists
os.makedirs(os.path.dirname(model_path), exist_ok=True)

# Function to download the model
def download_model(url, path):
    if not os.path.exists(path):
        print(f"Downloading model from {url}...")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        with open(path, 'wb') as file, tqdm(
            desc=path,
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(chunk_size=1024):
                file.write(data)
                bar.update(len(data))
        print("Model downloaded successfully.")
    else:
        print("Model already exists locally.")

# Download the model if it doesn't exist
download_model(model_url, model_path)

# Load the YOLOFace model
session = onnxruntime.InferenceSession(model_path)

def preprocess(frame, input_size):
    resized = cv2.resize(frame, input_size)
    normalized = resized / 255.0
    transposed = np.transpose(normalized, (2, 0, 1))
    input_tensor = np.expand_dims(transposed, axis=0).astype(np.float32)
    return input_tensor

def postprocess(detections, frame_shape, input_size, conf_threshold=0.5, nms_threshold=0.4):
    boxes, scores, landmarks = [], [], []
    frame_height, frame_width = frame_shape[:2]
    input_width, input_height = input_size

    for detection in detections:
        score = detection[4]
        if score > conf_threshold:
            center_x, center_y, width, height = detection[:4]
            x1 = int((center_x - width / 2) * frame_width / input_width)
            y1 = int((center_y - height / 2) * frame_height / input_height)
            x2 = int((center_x + width / 2) * frame_width / input_width)
            y2 = int((center_y + height / 2) * frame_height / input_height)
            boxes.append([x1, y1, x2, y2])
            scores.append(score)
            landmarks.append(detection[5:])

    indices = cv2.dnn.NMSBoxes(boxes, scores, conf_threshold, nms_threshold)
    if len(indices) > 0:
        indices = indices.flatten()
    unique_boxes = [boxes[i] for i in indices]
    unique_landmarks = [landmarks[i] for i in indices]
    return unique_boxes, unique_landmarks

def detect_faces(frame, input_size=(640, 640)):
    input_tensor = preprocess(frame, input_size)
    detections = session.run(None, {session.get_inputs()[0].name: input_tensor})[0]
    detections = np.squeeze(detections).T
    boxes, landmarks = postprocess(detections, frame.shape, input_size)
    return boxes, landmarks

def is_unique_face(face_encoding, known_faces, threshold=0.6):
    matches = face_recognition.compare_faces(known_faces, face_encoding, tolerance=threshold)
    return not any(matches)

def process_video(video_path, output_dir, margin=0.4):  # Increased margin to 0.7 (70%)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Error: Could not open video.")

    os.makedirs(output_dir, exist_ok=True)
    known_faces = []
    face_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        boxes, landmarks = detect_faces(frame)
        for box in boxes:
            x1, y1, x2, y2 = box

            # Calculate margin
            width = x2 - x1
            height = y2 - y1
            x1 = max(0, x1 - int(margin * width))
            y1 = max(0, y1 - int(margin * height))
            x2 = min(frame.shape[1], x2 + int(margin * width))
            y2 = min(frame.shape[0], y2 + int(margin * height))

            # Debug prints to verify margin adjustment
            print(f"Original box: ({box}), Margin: {margin}")
            print(f"Adjusted box: ({x1}, {y1}, {x2}, {y2})")

            face_img = frame[y1:y2, x1:x2]
            rgb_face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)

            # Get face encodings
            face_encodings = face_recognition.face_encodings(rgb_face_img)
            if face_encodings:
                face_encoding = face_encodings[0]

                if is_unique_face(face_encoding, known_faces):
                    known_faces.append(face_encoding)
                    face_count += 1
                    face_filename = os.path.join(output_dir, f'SRC_{face_count:02d}.jpg')
                    cv2.imwrite(face_filename, face_img, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
                    print(f"Saved {face_filename}")

    cap.release()
    cv2.destroyAllWindows()

# Example usage
if __name__ == "__main__":
    process_video('./demo_file/output_720p.mp4', './demo_file/multispecific', margin=0.4)  # Increased margin to 0.7 (70%)


In [ ]:
import sys
from options.test_options import TestOptions

# Temporarily clear sys.argv to avoid parsing any arguments
original_argv = sys.argv
sys.argv = [sys.argv[0]]  # Keep only the script name

# Create TestOptions and initialize
opt = TestOptions()
opt.initialize()
opt = opt.parse()  # This should return the parsed options

# Restore original sys.argv
sys.argv = original_argv

# Manually set or override the options
opt.multisepcific_dir = './demo_file/multispecific'
opt.video_path = './demo_file/output_720p.mp4'
opt.output_path = './output/out2.mp4'
opt.temp_path = './tmp'
opt.Arc_path = './arcface_model/arcface_checkpoint.tar'
opt.name = 'people'
opt.isTrain = False
opt.use_mask = True
opt.crop_size = 224  # Set this to the default value or the one you need

# Add any other options that are required by your script
# opt.other_option = value

crop_size = opt.crop_size

# Print out the options to verify
print(opt)

In [ ]:
import os
import cv2
import torch
import glob
from PIL import Image
import torch.nn.functional as F
from models.models import create_model
from options.test_options import TestOptions
from insightface_func.face_detect_crop_multi import Face_detect_crop

# Assuming opt is already defined and initialized
pic_specific = opt.pic_specific_path
crop_size = opt.crop_size
multisepcific_dir = opt.multisepcific_dir

torch.nn.Module.dump_patches = True
model = create_model(opt)
model.eval()

app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id=0, det_thresh=0.6, det_size=(640, 640))

# Directory to save images where face detection fails
error_dir = './error_images'
os.makedirs(error_dir, exist_ok=True)

# The specific person to be swapped (source)
source_specific_id_nonorm_list = []
source_path = os.path.join(multisepcific_dir, 'SRC_*')
source_specific_images_path = sorted(glob.glob(source_path))

with torch.no_grad():
    for source_specific_image_path in source_specific_images_path:
        specific_person_whole = cv2.imread(source_specific_image_path)
        if specific_person_whole is None:
            print(f"Error reading image: {source_specific_image_path}")
            continue

        result = app.get(specific_person_whole, crop_size)
        if result is None or len(result) != 2:
            print(f"Face detection failed for image: {source_specific_image_path}")
            error_image_path = os.path.join(error_dir, os.path.basename(source_specific_image_path))
            cv2.imwrite(error_image_path, specific_person_whole)
            continue

        specific_person_align_crop, _ = result
        specific_person_align_crop_pil = Image.fromarray(cv2.cvtColor(specific_person_align_crop[0], cv2.COLOR_BGR2RGB))
        specific_person = transformer_Arcface(specific_person_align_crop_pil)
        specific_person = specific_person.view(-1, specific_person.shape[0], specific_person.shape[1], specific_person.shape[2])
        specific_person = specific_person.cuda()
        specific_person_downsample = F.interpolate(specific_person, size=(112, 112))
        specific_person_id_nonorm = model.netArc(specific_person_downsample)
        source_specific_id_nonorm_list.append(specific_person_id_nonorm.clone())

# Print the count of successfully processed source images
print(f"Successfully processed source images: {len(source_specific_id_nonorm_list)}")

# The person who provides id information (list)
target_id_norm_list = []
target_path = os.path.join(multisepcific_dir, 'DST_*')
target_images_path = sorted(glob.glob(target_path))

with torch.no_grad():
    for target_image_path in target_images_path:
        img_a_whole = cv2.imread(target_image_path)
        if img_a_whole is None:
            print(f"Error reading image: {target_image_path}")
            continue

        result = app.get(img_a_whole, crop_size)
        if result is None or len(result) != 2:
            print(f"Face detection failed for image: {target_image_path}")
            error_image_path = os.path.join(error_dir, os.path.basename(target_image_path))
            cv2.imwrite(error_image_path, img_a_whole)
            continue

        img_a_align_crop, _ = result
        img_a_align_crop_pil = Image.fromarray(cv2.cvtColor(img_a_align_crop[0], cv2.COLOR_BGR2RGB))
        img_a = transformer_Arcface(img_a_align_crop_pil)
        img_id = img_a.view(-1, img_a.shape[0], img_a.shape[1], img_a.shape[2])
        img_id = img_id.cuda()
        img_id_downsample = F.interpolate(img_id, size=(112, 112))
        latend_id = model.netArc(img_id_downsample)
        latend_id = F.normalize(latend_id, p=2, dim=1)
        target_id_norm_list.append(latend_id.clone())

# Print the count of successfully processed target images
print(f"Successfully processed target images: {len(target_id_norm_list)}")

assert len(target_id_norm_list) == len(source_specific_id_nonorm_list), "The number of images in source and target directory must be same !!!"
video_swap(opt.video_path, target_id_norm_list, source_specific_id_nonorm_list, opt.id_thres, model, app, opt.output_path, temp_results_dir=opt.temp_path, no_simswaplogo=opt.no_simswaplogo, use_mask=opt.use_mask)